元のサンプルソース
--------------------------
https://github.com/thinkitcojp/TensorFlowDL-samples/blob/master/conv/mnist.py


CNN(Convolutional Neural Network)
---------------------------

- 多層パーセプトロンによる画像分類では、画像の特徴を抽出して分類したとはいえない
    - 画像を左上から操作してピクセルの白黒をみただけ
- 画像のフィルタ処理におけるフィルタの値を学習して特徴抽出を行うニューラルネットワーク 

In [3]:
#-*- coding:utf-8 -*-
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf

In [4]:
#mnistデータを格納したオブジェクトを呼び出す
mnist = input_data.read_data_sets("data/", one_hot=True)

"""モデル構築開始"""
# 入力データを定義
x = tf.placeholder(tf.float32, [None, 784])
# 入力データを画像として扱うために変換する
# 28 * 28 * 1チャンネル
img = tf.reshape(x,[-1,28,28,1])

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [5]:
# 畳み込み層1
# 正規分布で初期化
# この 5, 5は何だ? 縦:5 横5で畳込みして入力1の出力32チャンネルってことかな？
# [縦、横、チャネル数、フィルタ数（畳み込み後のチャンネル数）]
f1 = tf.Variable(tf.truncated_normal([5,5,1,32], stddev=0.1))
conv1 = tf.nn.conv2d(img, f1, strides=[1,1,1,1], padding='SAME')
b1 = tf.Variable(tf.constant(0.1, shape=[32]))
h_conv1 = tf.nn.relu(conv1+b1)

- 畳み込みニューラルネットワークのフィルタの値は普通のニューラルネットワークでいうところの重みにあたり、学習させる変数なので「tf.Variable」を使って宣言する
- 畳み込み層は「tf.nn.conv2d」で定義する
  - stridesは[バッチ方向, 縦方向, 横方向, チャンネル方向]を指す
  - strides[0]とstrides[3]は必ず1にする
  - そうしないと入力データをスキップしたり特定のチャンネルを無視してしまう
- paddingは元画像のまわりに画素値0のピクセルを用意する
    - 畳み込みを行うとフィルタの大きさとストライドが大きくなればなるほど畳み込み後の画像サイズは小さくなる
    - TensorFlowのpaddingはフィルタサイズにおける影響分を元画像に入れるかどうかを選択する
    - SAMEに設定するとパディングを入れて、「ストライドを1に設定すれば畳込み前後でも画像サイズは変わらなくなる」
    - VALIDに設定するとパディングは入らないため、畳込み後の画像サイズはフィルタサイズ分小さくなる

In [6]:
#プーリング層1
h_pool1 = tf.nn.max_pool(h_conv1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

- フィルタの中からもっとも大きい画素値を採用するマックスプーリングを用いている
    - 他には平均を取るプーリングなどがある
- ksizeはフィルタのサイズ [バッチ方向, 縦, 横, チャンネル]
    - 他の画像と一緒にプーリングしたり、異なるチャンネルでプーリングすることはないため、ksize[0], ksize[3]は1にする
- プーリングは画像を縮小することが目的
    - ここではストライドは2に設定してる(縦と横)
- paddingをSAMEにしているが、フィルタの大きさ分しかパディングしないため結果としてプーリング結果は画像のサイズが縦横ともに半分になる
- 畳込み、プーリングを得て28*28の1チャンネル画像が14*14の32チャンネルの画像になる

In [7]:
#畳み込み層2
f2 = tf.Variable(tf.truncated_normal([5,5,32,64], stddev=0.1))
conv2 = tf.nn.conv2d(h_pool1, f2, strides=[1,1,1,1], padding='SAME')
b2 = tf.Variable(tf.constant(0.1, shape=[64]))
h_conv2 = tf.nn.relu(conv2+b2)
#プーリング層2
h_pool2 = tf.nn.max_pool(h_conv2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

- さらに畳込みとプーリングをかけ 7 * 7の64チャンネルの画像にする

In [8]:
#畳み込まれているものをフラットな形に変換
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])

- 最終層でソフトマックス関数を用いて[バッチサイズ, 画像の種類]の形に2階Tensorにするために、tf.reshapeを用いて4階テンソルを2階テンソルに変換する

In [9]:
#全結合層
w_fc1 = tf.Variable(tf.truncated_normal([7*7*64, 1024], stddev=0.1))
b_fc1 = tf.Variable(tf.constant(0.1, shape=[1024]))
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, w_fc1) + b_fc1)

In [10]:
#出力層
w_fc2 = tf.Variable(tf.truncated_normal([1024, 10], stddev=0.1))
b_fc2 = tf.Variable(tf.constant(0.1, shape=[10]))
out = tf.nn.softmax(tf.matmul(h_fc1, w_fc2) + b_fc2)

In [11]:
#正解データの型を定義
y = tf.placeholder(tf.float32, [None, 10])
#誤差関数（クロスエントロピー）
loss = tf.reduce_mean(-tf.reduce_sum(y * tf.log(out + 1e-5), axis=[1]))

- 微少数を加算しないとlogを計算したときに無限大の値をとりニューラルネットワークが発散するため必ず微小量を加えておく

In [12]:
#訓練
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(loss)

In [13]:
#評価
correct = tf.equal(tf.argmax(out,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [15]:
init =tf.global_variables_initializer()

In [16]:
"""実行部分"""
with tf.Session() as sess:
    sess.run(init)
    #テストデータをロード
    test_images = mnist.test.images
    test_labels = mnist.test.labels

    for step in range(1000):
        train_images, train_labels = mnist.train.next_batch(50)
        sess.run(train_step, feed_dict={x:train_images ,y:train_labels})

        #10階ごとに精度を検証
        if step % 100 == 0:
            acc_val = sess.run( accuracy, feed_dict={x:test_images, y:test_labels})
            print('Step %d: accuracy = %.2f' % (step, acc_val))

Step 0: accuracy = 0.14
Step 100: accuracy = 0.90
Step 200: accuracy = 0.94
Step 300: accuracy = 0.94
Step 400: accuracy = 0.94
Step 500: accuracy = 0.96
Step 600: accuracy = 0.96
Step 700: accuracy = 0.96
Step 800: accuracy = 0.96
Step 900: accuracy = 0.97
